In [29]:
# Loading libs
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff
# Start a Spark Session
spark = SparkSession.builder.appName("UsedCarSalesData").getOrCreate()

##### Spark Join

In [30]:
df = spark.read.json("raw data/new_listing_*.json")
df.show()

+------+-------------------+-------------------+-----+--------+-----+------+----+----+
| Brand|        Date listed|       Date removed|Model|Odometer|Price|  Rego|Trim|Year|
+------+-------------------+-------------------+-----+--------+-----+------+----+----+
|Toyota|2020-05-19 00:00:00|2021-01-27 00:00:00|    B|  196084|90277|LHZ303|   2|2020|
|Nissan|2020-08-14 00:00:00|2020-08-24 00:00:00|    A|  188788|50621|NKQ284|   2|2020|
| Mazda|2018-01-17 00:00:00|2019-01-14 00:00:00|    A|  148494|56404|OYH081|   2|2018|
| Mazda|2021-11-13 00:00:00|2022-08-28 00:00:00|    C|   52619|26233|DBO484|   3|2021|
| Mazda|2021-11-28 00:00:00|2022-02-22 00:00:00|    D|   80540|43792|YKO883|   1|2021|
|   BMW|2021-01-18 00:00:00|2021-11-24 00:00:00|    B|  189321|96185|RSI941|   2|2021|
|   BMW|2019-03-14 00:00:00|2019-08-05 00:00:00|    D|  166892|55935|VGB787|   2|2019|
|   BMW|2017-02-08 00:00:00|2017-07-31 00:00:00|    A|   17832|62276|SRT961|   1|2017|
|   BMW|2019-07-15 00:00:00|2019-11-16 00:0

#### Spark Transformation

In [31]:
# Add the "turnover" column with the calculation of [Date removed] - [Date listed] in days
df = df.withColumn("Turnover", datediff(col("Date removed"), col("Date listed")).cast("int"))

In [32]:
df.show()
df.dtypes

+------+-------------------+-------------------+-----+--------+-----+------+----+----+--------+
| Brand|        Date listed|       Date removed|Model|Odometer|Price|  Rego|Trim|Year|Turnover|
+------+-------------------+-------------------+-----+--------+-----+------+----+----+--------+
|Toyota|2020-05-19 00:00:00|2021-01-27 00:00:00|    B|  196084|90277|LHZ303|   2|2020|     253|
|Nissan|2020-08-14 00:00:00|2020-08-24 00:00:00|    A|  188788|50621|NKQ284|   2|2020|      10|
| Mazda|2018-01-17 00:00:00|2019-01-14 00:00:00|    A|  148494|56404|OYH081|   2|2018|     362|
| Mazda|2021-11-13 00:00:00|2022-08-28 00:00:00|    C|   52619|26233|DBO484|   3|2021|     288|
| Mazda|2021-11-28 00:00:00|2022-02-22 00:00:00|    D|   80540|43792|YKO883|   1|2021|      86|
|   BMW|2021-01-18 00:00:00|2021-11-24 00:00:00|    B|  189321|96185|RSI941|   2|2021|     310|
|   BMW|2019-03-14 00:00:00|2019-08-05 00:00:00|    D|  166892|55935|VGB787|   2|2019|     144|
|   BMW|2017-02-08 00:00:00|2017-07-31 0

[('Brand', 'string'),
 ('Date listed', 'string'),
 ('Date removed', 'string'),
 ('Model', 'string'),
 ('Odometer', 'bigint'),
 ('Price', 'bigint'),
 ('Rego', 'string'),
 ('Trim', 'bigint'),
 ('Year', 'bigint'),
 ('Turnover', 'int')]

In [33]:
from pyspark.sql.types import StringType, IntegerType, ShortType
from pyspark.sql.types import StringType
from pyspark.sql.functions import date_format

df = df.withColumn("Date listed", date_format(df["Date listed"], "yyyy-MM-dd"))
df = df.withColumn("Date removed", date_format(df["Date removed"], "yyyy-MM-dd"))
df = df.withColumn("Rego", df["Rego"].cast(StringType()))
df = df.withColumn("Brand", df["Brand"].cast(StringType()))
df = df.withColumn("Model", df["Model"].cast(StringType()))
df = df.withColumn("Trim", df["Trim"].cast(StringType()))
df = df.withColumn("Year", df["Year"].cast(ShortType()))
df = df.withColumn("Odometer", df["Odometer"].cast(IntegerType()))
df = df.withColumn("Turnover", df["Turnover"].cast(ShortType()))


In [34]:
df.show()
df.dtypes

+------+-----------+------------+-----+--------+-----+------+----+----+--------+
| Brand|Date listed|Date removed|Model|Odometer|Price|  Rego|Trim|Year|Turnover|
+------+-----------+------------+-----+--------+-----+------+----+----+--------+
|Toyota| 2020-05-19|  2021-01-27|    B|  196084|90277|LHZ303|   2|2020|     253|
|Nissan| 2020-08-14|  2020-08-24|    A|  188788|50621|NKQ284|   2|2020|      10|
| Mazda| 2018-01-17|  2019-01-14|    A|  148494|56404|OYH081|   2|2018|     362|
| Mazda| 2021-11-13|  2022-08-28|    C|   52619|26233|DBO484|   3|2021|     288|
| Mazda| 2021-11-28|  2022-02-22|    D|   80540|43792|YKO883|   1|2021|      86|
|   BMW| 2021-01-18|  2021-11-24|    B|  189321|96185|RSI941|   2|2021|     310|
|   BMW| 2019-03-14|  2019-08-05|    D|  166892|55935|VGB787|   2|2019|     144|
|   BMW| 2017-02-08|  2017-07-31|    A|   17832|62276|SRT961|   1|2017|     173|
|   BMW| 2019-07-15|  2019-11-16|    B|   55876|76508|SAW182|   3|2019|     124|
|   BMW| 2021-05-20|  2021-0

[('Brand', 'string'),
 ('Date listed', 'string'),
 ('Date removed', 'string'),
 ('Model', 'string'),
 ('Odometer', 'int'),
 ('Price', 'bigint'),
 ('Rego', 'string'),
 ('Trim', 'string'),
 ('Year', 'smallint'),
 ('Turnover', 'smallint')]

#### Write the original CSV data into Parquet format

In [ ]:
df.write.parquet("silver data")